<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

# Data Analysis with Text AI

In this notebook, we will do some basic analysis on the semantic data extracted from the Customer Support Tickets.

We will be running SQL queries using both the <a href="https://github.com/ploomber/jupysql" target="_blank" rel="noopener">JupySQL</a>SQL Magic
and the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a>.

## Prerequisites

Prior to using this notebook the following step needs to be completed:
1. [Run Text AI Preprocessing](txaie_preprocessing.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Initialise JupySQL
Let's bring up JupySQL and connect to the database via SQLAlchemy. Please refer to the documentation of <a href="https://github.com/exasol/sqlalchemy-exasol" target="_blank" rel="noopener">sqlalchemy-exasol</a> for details on how to connect to the database using the Exasol SQLAlchemy driver.

In [ ]:
%run ../utils/jupysql_init.ipynb

### Import Python modules

In [ ]:
import pandas as pd
from ipywidgets import Dropdown
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from exasol.nb_connector.connections import open_pyexasol_connection

## Urgent vs Non-urgent tieckts

### Create `TICKET_URGENCY` view

First, let us create a view that adds the `IS_URGENT` flag to each document. For this, we can use the results of the topic classifier. We have run the classifier using two topics - "urgent" and "not urgent". The output of the classifier is the cross product of the documents (tickets) and topics. For our purpose we can select the rows with the "urgent" topic.

For each document, the classifier sets the rank and the score of every topic. We do not want to rely purely on the rank, because it does not indicate certainty of the classification. Instead we will use the score which is akin a probability of the document belonging to the given topic (assuming the topics are mutually exclusive). We will deem the ticket as urgent if its "urgent" topic score is greater than 0.7.

In [ ]:
%%sql
CREATE OR REPLACE VIEW TICKET_URGENCY AS
SELECT 
    D.*,
    T.TOPIC_SCORE,
    T.TOPIC_SCORE > 0.7 AS IS_URGENT
FROM DOCUMENTS_AI_LAB_CUSTOMER_SUPPORT_TICKETS_VIEW_VIEW D
JOIN TOPICS_VIEW T ON 
    D.TEXT_DOC_ID=T.TEXT_DOC_ID AND 
    D.TEXT_CHAR_BEGIN=T.TEXT_CHAR_BEGIN AND
    D.TEXT_CHAR_END=T.TEXT_CHAR_END
WHERE T.TOPIC='urgent'

Now we can inspect the urgent tickets. 

**Note**: We will often see tickets that contain phrases, like "I need them urgently", or "but the issue remains unresolved". These prhases indicate for the model a higher urgency of a ticket. However, this might only reflect the urgency for the author of the ticket and are soley based on the content of ticket description. The extracted urgency might not reflect your internal work priority for tickets. Also, for the classification of the ticket priority, factors that are external to the text might need to be considered. The model here can only consider the information conveyed by the text which might be able to give input for the final classification of the ticket priority. 

In [ ]:
%%sql
SELECT TICKET_ID, CUSTOMER_NAME, DATE_OF_PURCHASE, TICKET_DESCRIPTION, TOPIC_SCORE
FROM TICKET_URGENCY
WHERE IS_URGENT = TRUE

### Count urgent and non-urgent tickets

Here we will display the total number of urgent and non-urgent tickets.

In [ ]:
%%sql --save URGENT_TICKET_COUNT
SELECT 'Urgent' as URGENCY, SUM(IS_URGENT) AS CNT
FROM TICKET_URGENCY
UNION
SELECT 'Not Urgent' as URGENCY, SUM(1 - IS_URGENT) AS CNT
FROM TICKET_URGENCY
ORDER BY 1

In [ ]:
%sqlplot pie --table URGENT_TICKET_COUNT --column URGENCY CNT

## Products with urgent tickets

In this section we will take a look at which products have the biggest number of urgent tickets.

### Inspecting the Products

Let us first inspect the products that were extracted from the tickets

We will use the results of the named entity extractor. The products are entities where the `entity_type` looks like "product_xxx". Note, that the extractor also provides the positions of entities in the text, which we don't need here. We shall account for the fact that there are potentially multiple instances of the same entity in a document.                                                                                                                                           

In [ ]:
%%sql products_with_ticket_count <<
SELECT E.ENTITY as PRODUCT, COUNT(DISTINCT D.TICKET_ID) as TICKET_COUNT
FROM DOCUMENTS_AI_LAB_CUSTOMER_SUPPORT_TICKETS_VIEW_VIEW D
JOIN NAMED_ENTITY_VIEW E ON 
    D.TEXT_DOC_ID=E.TEXT_DOC_ID AND 
    D.TEXT_CHAR_BEGIN=E.TEXT_CHAR_BEGIN AND
    D.TEXT_CHAR_END=E.TEXT_CHAR_END
WHERE E.ENTITY_TYPE like 'product%'
GROUP BY E.ENTITY
ORDER BY TICKET_COUNT DESC

Now lets show the products in a word cloud. The font size of the product is larger for products that occur in more tickets. The colors in the word cloud are choosen to easier distingish different words.

In [ ]:
products_with_ticket_count_df = products_with_ticket_count.DataFrame()
products_with_ticket_count_dict = products_with_ticket_count_df.set_index("product")["ticket_count"].to_dict()
wordcloud = WordCloud(background_color="white").generate_from_frequencies(products_with_ticket_count_dict)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Create `PRODUCT_ATTENTION` view

From this point onward we will be looking at a limited number of products with the biggest number of urgent tickets. Let us first create a view `URGENT_PRODUCT_CO_OCCURRENCE` that selects products that are contained inside extracted topic with the label `urgent`.

To create `URGENT_PRODUCT_CO_OCCURRENCE` view we are using the `CO_OCCURRENCE` view which is created by the `StandardExtractor`. The `CO_OCCURRENCE` view joins topics, named entities and keywords in the same document together and provides addtional columns to define the positional relations between them, such as the distance between an named entity and a keyword. However, for the moment we don not need these relations

In [ ]:
%%sql
CREATE OR REPLACE VIEW URGENT_PRODUCT_CO_OCCURRENCE AS
SELECT *
FROM CO_OCCURRENCE COO
WHERE TOPIC = 'urgent'
AND ENTITY_TYPE like 'product%'

Now we can create a view `PRODUCT_ATTENTION` on top of `URGENT_PRODUCT_CO_OCCURRENCE`, that selects the ten products with the most urgent tickets.

In [ ]:
%%sql
CREATE OR REPLACE VIEW PRODUCT_ATTENTION AS
SELECT COO.ENTITY AS PRODUCT, COUNT(DISTINCT D.TICKET_ID) as URGENT_TICKETS
FROM URGENT_PRODUCT_CO_OCCURRENCE COO
JOIN DOCUMENTS_AI_LAB_CUSTOMER_SUPPORT_TICKETS_VIEW_VIEW AS D
ON D.text_doc_id = COO.text_doc_id
WHERE COO.TOPIC_SCORE > 0.7
GROUP BY local.PRODUCT
ORDER BY local.URGENT_TICKETS DESC
LIMIT 10

### Show the number of tickets for the selected products

Here we will look at the number of urgent and non-urgent tickets for each of the selected products.

In [ ]:
%%sql
CREATE OR REPLACE VIEW TICKET_URGENCY_FOR_PRODUCTS_UNDER_ATTENTION AS
SELECT DISTINCT TICKET_ID, ENTITY AS PRODUCT, TOPIC_SCORE > 0.7 as IS_URGENT
FROM URGENT_PRODUCT_CO_OCCURRENCE COO
JOIN DOCUMENTS_AI_LAB_CUSTOMER_SUPPORT_TICKETS_VIEW_VIEW AS D
ON D.text_doc_id = COO.text_doc_id
JOIN PRODUCT_ATTENTION A 
ON COO.ENTITY=A.PRODUCT

In [ ]:
%%sql
SELECT PRODUCT, SUM(IS_URGENT) as IS_URGENT
FROM TICKET_URGENCY_FOR_PRODUCTS_UNDER_ATTENTION
GROUP BY PRODUCT
ORDER BY IS_URGENT DESC

Now let us compute the percentage of urgent and non-urgent tickets for each of the selected products and plot it as a bar chart.

In [ ]:
%%sql precent_of_urgent_tickets_per_product <<
SELECT PRODUCT, SUM(1-IS_URGENT)/COUNT(1)*100 as IS_NOT_URGENT, SUM(IS_URGENT)/COUNT(1)*100 as IS_URGENT
FROM TICKET_URGENCY_FOR_PRODUCTS_UNDER_ATTENTION
GROUP BY PRODUCT
ORDER BY IS_URGENT ASC

In [ ]:
precent_of_urgent_tickets_per_product.DataFrame().set_index("product").plot(kind='barh', stacked=True, xlabel="percent of tickets")

## Reasons behind tickets

In this section we will try to look at common reasons for raising a ticket, in a scope of a particular product. The reasons or clues, as it might be a better term, are represented by keywords. 

First we will load out selected products into a dropdown box.

In [ ]:
query = """
SELECT PRODUCT
FROM PRODUCT_ATTENTION
ORDER BY URGENT_TICKETS DESC
"""

with open_pyexasol_connection(ai_lab_config, compression=True, schema=ai_lab_config.db_schema) as connection:
    options = connection.execute(query=query).fetchcol()

dropdown = Dropdown(
    options=options,
    style={'description_width': 'initial'}
)
display(dropdown)

### View keywords in a Word Cloud

We will compute frequencies of the keywords in the tickets for the selected product. There are few keywords we would want to exclude. Firstly, this is the name of the product or any part of the name. Then we also want to exclude some common words which, unfortunately are often identified as keywords. Below is the list of such common words. It can be extended as appropriate.

In [ ]:
exclude_keywords = "('product', 'issue')"

Now we are ready to compute and display the keyword frequencies. 

We will use the sum of the keyword scores as a proxy for the frequency. The score is akin a probability that the word is actually a keyword. So, the sum of the scores can be interpreted as an expectation of the total number of occurrences of the given keyword in the collection of tickets for the selected product.
                                                                                                                                                            Note, that the keyword extractor output includes the positions of keywords in the text. These positions themselves are not required for our analysis. However, the implication is that a document may have multiple instances of the same keyword. For the sake of simplicity, we compute the frequency not limiting the number of occurrences of a keyword in a single document to one.                                                                                                                               
We will reuse our `URGENT_PRODUCT_CO_OCCURRENCE` view from before. We only add addtional conditions and aggregate the data as described.

In [ ]:
%%sql product_keywords_with_scores <<
SELECT
    COO.ENTITY as PRODUCT,
    COO.KEYWORD,
    SUM(COO.KEYWORD_SCORE) AS SCORE
FROM URGENT_PRODUCT_CO_OCCURRENCE COO
WHERE COO.ENTITY='{{dropdown.value}}'
AND INSTR('{{dropdown.value.strip().upper()}}', UCASE(COO.KEYWORD)) = 0
AND COO.KEYWORD NOT IN {{exclude_keywords}}
GROUP BY COO.ENTITY, COO.KEYWORD
ORDER BY SCORE DESC

We will plot the the keywords in a word cloud. The font-size will be larger for keywords that have a higher sum of scores.

In [ ]:
keywords_with_scores = product_keywords_with_scores.DataFrame()[["keyword", "score"]]
keywords_with_scores_dict = keywords_with_scores.set_index("keyword")["score"].to_dict()
wordcloud = WordCloud(background_color="white").generate_from_frequencies(keywords_with_scores_dict)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()